In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
# 加载数据
df = pd.read_csv('zhengzhou_house_data.csv',encoding='UTF-8-SIG')

# 查看数据的前几行
df.head()

,name,rooms,floor,size,price,unit_price,features,orientation,floor_height
0,中间好楼层 冯庄地铁口 人车分流小区 装修保持好拎包入住,"['2', '室', '2', '厅', '1', '卫']",中层(共33层),88.55㎡,79.9万,9024元/㎡,"['南', '优势户型', '房东急售', '随时可看']",南,33
1,亚星城市山水 交通便捷 双卫生间 临地铁 拎包入住,"['3', '室', '2', '厅', '2', '卫']",中层(共28层),131.62㎡,118万,8966元/㎡,"['南', '满五年', '近地铁', '车位充足']",南,28
2,2024年交付 121平精装修 双卫三房 拎包入住 看房随时,"['3', '室', '2', '厅', '2', '卫']",低层(共20层),121.56㎡,128万,10530元/㎡,"['南', '热门小区', '南北通透', '满五']",南,20
3,铁道陇海家园 近地铁 精装可拎包入住 商圈成熟 南北通透,"['3', '室', '2', '厅', '2', '卫']",高层(共26层),124.86㎡,150万,12014元/㎡,"['南', '满五年', '近地铁', '采光较好']",南,26
4,必看好房 业主二次装修 拎包入住 近 出门就是地铁口,"['3', '室', '2', '厅', '1', '卫']",高层(共33层),94.41㎡,96万,10169元/㎡,"['南', '热门小区', '满五', '房东急售']",南,33


In [3]:
# 提取数值部分并转换为数值类型
df['size'] = df['size'].str.replace('㎡', '').astype(float)  # 去掉㎡，转换为浮点数
df['price'] = df['price'].str.replace('万', '').astype(float) * 10000  # 去掉万，转换为实际数值
df['unit_price'] = df['unit_price'].str.replace('元/㎡', '').astype(float)  # 去掉元/㎡，转换为浮点数

# 修改列名，添加单位
df.rename(columns={
    'size': 'size (㎡)',
    'price': 'price (元)',
    'unit_price': 'unit_price (元/㎡)'  # 正确的列名修改方式
}, inplace=True)

# 打印处理后的数据
print("\n处理后的数据：")
print(df)


处理后的数据：
                                 name                           rooms  \
0        中间好楼层 冯庄地铁口 人车分流小区 装修保持好拎包入住  ['2', '室', '2', '厅', '1', '卫']   
1           亚星城市山水 交通便捷 双卫生间 临地铁 拎包入住  ['3', '室', '2', '厅', '2', '卫']   
2      2024年交付 121平精装修 双卫三房 拎包入住 看房随时  ['3', '室', '2', '厅', '2', '卫']   
3        铁道陇海家园 近地铁 精装可拎包入住 商圈成熟 南北通透  ['3', '室', '2', '厅', '2', '卫']   
4        必看好房 业主二次装修 拎包入住 近   出门就是地铁口  ['3', '室', '2', '厅', '1', '卫']   
...                               ...                             ...   
12084       近地铁 房龄新 小三房 医疗配套成熟 有电梯 朝南  ['3', '室', '2', '厅', '1', '卫']   
12085       不临高架 采光三房 全屋断桥铝窗户 精装 和昌澜景  ['3', '室', '2', '厅', '1', '卫']   
12086      风和日丽家园 满五 楼层好 商品房 正南朝向 楼龄新  ['2', '室', '2', '厅', '1', '卫']   
12087       满五   南向采光好 医疗配套成熟 带电梯 楼龄新  ['3', '室', '2', '厅', '1', '卫']   
12088     满五   高楼层视野好 医疗配套成熟 购物方便 近地铁  ['3', '室', '2', '厅', '1', '卫']   

          floor  size (㎡)  price (元)  unit_price (元/㎡)  \
0      中层(共33层)     88.55   799000.0            9024.0  

In [4]:
# 检查缺失值
df.isnull().sum()

# 填充缺失值（用均值填充数值列，用众数填充分类列）
df['size (㎡)'].fillna(df['size (㎡)'].mean(), inplace=True)  # 用面积的均值填充缺失值
df['price (元)'].fillna(df['price (元)'].mean(), inplace=True)  # 用总价的均值填充缺失值
df['unit_price (元/㎡)'].fillna(df['unit_price (元/㎡)'].mean(), inplace=True)  # 用单价的均值填充缺失值
df['floor_height'].fillna(df['floor_height'].mean(), inplace=True)  # 用层高的均值填充缺失值
df['name'].fillna(df['name'].mode()[0], inplace=True)  # 用小区名称的众数填充缺失值
df['rooms'].fillna(df['rooms'].mode()[0], inplace=True)  # 用户型的众数填充缺失值
df['features'].fillna(df['features'].mode()[0], inplace=True)  # 用特征的众数填充缺失值
df['orientation'].fillna(df['orientation'].mode()[0], inplace=True)  # 用朝向的众数填充缺失值

In [5]:
# 删除重复行
df.drop_duplicates(inplace=True)

In [6]:
import pandas as pd
import ast
from sklearn.preprocessing import StandardScaler


# 确保 'rooms' 列是真正的列表格式
def convert_to_list(room_str):
    try:
        # 尝试将字符串转换为列表
        return ast.literal_eval(room_str)
    except:
        # 如果转换失败，返回原始值（可能是错误数据）
        return room_str

# 应用转换函数
df['rooms'] = df['rooms'].apply(convert_to_list)

# 检查转换后的数据
print("转换后的 'rooms' 列：")
print(df['rooms'].head())

# 提取房间数量的函数
def extract_room_info(room_list):
    try:
        # 提取卧室、客厅和卫生间数量
        bedroom = int(room_list[0])  # 卧室数量
        living_room = int(room_list[2])  # 客厅数量
        bathroom = int(room_list[4])  # 卫生间数量
        return bedroom, living_room, bathroom
    except Exception as e:
        # 如果提取失败，打印错误信息并返回默认值
        print(f"Error in extract_room_info: {e}")
        return None, None, None
# 应用函数并创建新列
df[['bedroom', 'living_room', 'bathroom']] = df['rooms'].apply(lambda x: pd.Series(extract_room_info(x)))
# 删除原始 'rooms' 列
df.drop(columns=['rooms'], inplace=True)

# # 特征缩放
# scaler = StandardScaler()
# df[['bedroom', 'living_room', 'bathroom']] = scaler.fit_transform(df[['bedroom', 'living_room', 'bathroom']])

# # 查看结果
# print("处理后的数据：")
# print(df.head())

转换后的 'rooms' 列：
0    [2, 室, 2, 厅, 1, 卫]
1    [3, 室, 2, 厅, 2, 卫]
2    [3, 室, 2, 厅, 2, 卫]
3    [3, 室, 2, 厅, 2, 卫]
4    [3, 室, 2, 厅, 1, 卫]
Name: rooms, dtype: object


In [7]:
print(df.columns)

Index(['name', 'floor', 'size (㎡)', 'price (元)', 'unit_price (元/㎡)',
       'features', 'orientation', 'floor_height', 'bedroom', 'living_room',
       'bathroom'],
      dtype='object')


In [8]:
import pandas as pd

# 定义一个函数来判断户型大小
def classify_room(row):
    bedroom = row["bedroom"]
    living_room = row["living_room"]
    bathroom = row["bathroom"]
    
    # 定义大户型和小户型的规则
    if bedroom >= 3 or living_room >= 2 or bathroom >= 2:
        return "大户型"
    else:
        return "小户型"

# 应用函数到每一行
df["户型分类"] = df.apply(classify_room, axis=1)

# 查看结果
print(df)


                                 name     floor  size (㎡)  price (元)  \
0        中间好楼层 冯庄地铁口 人车分流小区 装修保持好拎包入住  中层(共33层)     88.55   799000.0   
1           亚星城市山水 交通便捷 双卫生间 临地铁 拎包入住  中层(共28层)    131.62  1180000.0   
2      2024年交付 121平精装修 双卫三房 拎包入住 看房随时  低层(共20层)    121.56  1280000.0   
3        铁道陇海家园 近地铁 精装可拎包入住 商圈成熟 南北通透  高层(共26层)    124.86  1500000.0   
4        必看好房 业主二次装修 拎包入住 近   出门就是地铁口  高层(共33层)     94.41   960000.0   
...                               ...       ...       ...        ...   
12084       近地铁 房龄新 小三房 医疗配套成熟 有电梯 朝南  高层(共30层)     89.00   930000.0   
12085       不临高架 采光三房 全屋断桥铝窗户 精装 和昌澜景  中层(共34层)     88.63   900000.0   
12086      风和日丽家园 满五 楼层好 商品房 正南朝向 楼龄新  中层(共33层)     77.00  1090000.0   
12087       满五   南向采光好 医疗配套成熟 带电梯 楼龄新  中层(共27层)    112.46  1550000.0   
12088     满五   高楼层视野好 医疗配套成熟 购物方便 近地铁  高层(共22层)    102.98  1260000.0   

       unit_price (元/㎡)                       features orientation  \
0                9024.0  ['南', '优势户型', '房东急售', '随时可看']           

In [9]:
print(df.columns)

Index(['name', 'floor', 'size (㎡)', 'price (元)', 'unit_price (元/㎡)',
       'features', 'orientation', 'floor_height', 'bedroom', 'living_room',
       'bathroom', '户型分类'],
      dtype='object')


In [10]:
# 删除floor列
df.drop(columns=["floor"], inplace=True)
# 将floor_height列重命名为floor
df.rename(columns={"floor_height": "floor"}, inplace=True)

df.rename(columns={'户型分类': 'Apartment Layout Classification'}, inplace=True)

In [11]:
print(df.columns)

Index(['name', 'size (㎡)', 'price (元)', 'unit_price (元/㎡)', 'features',
       'orientation', 'floor', 'bedroom', 'living_room', 'bathroom',
       'Apartment Layout Classification'],
      dtype='object')


In [12]:
# 创建新列 price_10k_rmb，将 price（元）除以 10000
df['price_10k_rmb'] = df['price (元)'] / 10000

# 查看转换后的数据
print(df)

                                 name  size (㎡)  price (元)  unit_price (元/㎡)  \
0        中间好楼层 冯庄地铁口 人车分流小区 装修保持好拎包入住     88.55   799000.0            9024.0   
1           亚星城市山水 交通便捷 双卫生间 临地铁 拎包入住    131.62  1180000.0            8966.0   
2      2024年交付 121平精装修 双卫三房 拎包入住 看房随时    121.56  1280000.0           10530.0   
3        铁道陇海家园 近地铁 精装可拎包入住 商圈成熟 南北通透    124.86  1500000.0           12014.0   
4        必看好房 业主二次装修 拎包入住 近   出门就是地铁口     94.41   960000.0           10169.0   
...                               ...       ...        ...               ...   
12084       近地铁 房龄新 小三房 医疗配套成熟 有电梯 朝南     89.00   930000.0           10450.0   
12085       不临高架 采光三房 全屋断桥铝窗户 精装 和昌澜景     88.63   900000.0           10155.0   
12086      风和日丽家园 满五 楼层好 商品房 正南朝向 楼龄新     77.00  1090000.0           14156.0   
12087       满五   南向采光好 医疗配套成熟 带电梯 楼龄新    112.46  1550000.0           13783.0   
12088     满五   高楼层视野好 医疗配套成熟 购物方便 近地铁    102.98  1260000.0           12236.0   

                            features or

In [13]:
# 修改列名
df.rename(columns={'size (㎡)': 'size_m2'}, inplace=True)

# 将 unit_price (元/㎡)的值除以 10000 并修改列名
df['unit_price_10k_rmb_per_m2'] = df['unit_price (元/㎡)'] / 10000


# 查看转换后的数据
print(df)

                                 name  size_m2  price (元)  unit_price (元/㎡)  \
0        中间好楼层 冯庄地铁口 人车分流小区 装修保持好拎包入住    88.55   799000.0            9024.0   
1           亚星城市山水 交通便捷 双卫生间 临地铁 拎包入住   131.62  1180000.0            8966.0   
2      2024年交付 121平精装修 双卫三房 拎包入住 看房随时   121.56  1280000.0           10530.0   
3        铁道陇海家园 近地铁 精装可拎包入住 商圈成熟 南北通透   124.86  1500000.0           12014.0   
4        必看好房 业主二次装修 拎包入住 近   出门就是地铁口    94.41   960000.0           10169.0   
...                               ...      ...        ...               ...   
12084       近地铁 房龄新 小三房 医疗配套成熟 有电梯 朝南    89.00   930000.0           10450.0   
12085       不临高架 采光三房 全屋断桥铝窗户 精装 和昌澜景    88.63   900000.0           10155.0   
12086      风和日丽家园 满五 楼层好 商品房 正南朝向 楼龄新    77.00  1090000.0           14156.0   
12087       满五   南向采光好 医疗配套成熟 带电梯 楼龄新   112.46  1550000.0           13783.0   
12088     满五   高楼层视野好 医疗配套成熟 购物方便 近地铁   102.98  1260000.0           12236.0   

                            features orientation  f

In [14]:
# 删除旧列
df.drop(columns=["price (元)"], inplace=True)
df.drop(columns=["unit_price (元/㎡)"], inplace=True)

print(df)

                                 name  size_m2                       features  \
0        中间好楼层 冯庄地铁口 人车分流小区 装修保持好拎包入住    88.55  ['南', '优势户型', '房东急售', '随时可看']   
1           亚星城市山水 交通便捷 双卫生间 临地铁 拎包入住   131.62    ['南', '满五年', '近地铁', '车位充足']   
2      2024年交付 121平精装修 双卫三房 拎包入住 看房随时   121.56    ['南', '热门小区', '南北通透', '满五']   
3        铁道陇海家园 近地铁 精装可拎包入住 商圈成熟 南北通透   124.86    ['南', '满五年', '近地铁', '采光较好']   
4        必看好房 业主二次装修 拎包入住 近   出门就是地铁口    94.41    ['南', '热门小区', '满五', '房东急售']   
...                               ...      ...                            ...   
12084       近地铁 房龄新 小三房 医疗配套成熟 有电梯 朝南    89.00                   ['南满五年采光较好']   
12085       不临高架 采光三房 全屋断桥铝窗户 精装 和昌澜景    88.63                   ['南满五年采光较好']   
12086      风和日丽家园 满五 楼层好 商品房 正南朝向 楼龄新    77.00                    ['南满五年近地铁']   
12087       满五   南向采光好 医疗配套成熟 带电梯 楼龄新   112.46                    ['南满五年近地铁']   
12088     满五   高楼层视野好 医疗配套成熟 购物方便 近地铁   102.98                    ['南满五年近地铁']   

      orientation  floor  b

In [15]:
df.to_csv('zhengzhou_house_data.csv', index=False, encoding='utf-8-sig')

print("数据清洗完成并已保存到zhengzhou_house_data.csv 文件中")

数据清洗完成并已保存到zhengzhou_house_data.csv 文件中
